In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os

2023-02-25 15:55:18.580478: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 15:55:18.658746: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-25 15:55:19.027516: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/thomas/miniconda3/lib/:/home/thomas/miniconda3/lib/:/home/thomas/miniconda3/envs/tf/lib/
20

In [2]:
def normalize(df):
    return (df/df.max())

In [3]:
CompleteTrainingDF = pd.read_csv('Data/train.csv')

In [4]:
TrainingDF = CompleteTrainingDF.loc[:,["Survived","SibSp","Parch","Fare"]]
TrainingDF = pd.concat([TrainingDF,pd.get_dummies(CompleteTrainingDF["Embarked"])],axis=1)
TrainingDF = pd.concat([TrainingDF,pd.get_dummies(CompleteTrainingDF["Sex"])],axis=1)
TrainingDF = pd.concat([TrainingDF,pd.get_dummies(CompleteTrainingDF["Pclass"],prefix="Class")],axis=1)
TrainingDF["Fare"] = normalize(TrainingDF["Fare"])

In [5]:
print(TrainingDF)

     Survived  SibSp  Parch      Fare  C  Q  S  female  male  Class_1  \
0           0      1      0  0.014151  0  0  1       0     1        0   
1           1      1      0  0.139136  1  0  0       1     0        1   
2           1      0      0  0.015469  0  0  1       1     0        0   
3           1      1      0  0.103644  0  0  1       1     0        1   
4           0      0      0  0.015713  0  0  1       0     1        0   
..        ...    ...    ...       ... .. .. ..     ...   ...      ...   
886         0      0      0  0.025374  0  0  1       0     1        0   
887         1      0      0  0.058556  0  0  1       1     0        1   
888         0      1      2  0.045771  0  0  1       1     0        0   
889         1      0      0  0.058556  1  0  0       0     1        1   
890         0      0      0  0.015127  0  1  0       0     1        0   

     Class_2  Class_3  
0          0        1  
1          0        0  
2          0        1  
3          0        0  
4  

In [6]:
xtrain = TrainingDF.iloc[:600,1:] #Trims off ytrain
ytrain = TrainingDF.iloc[:600,:1] #Trims off ytrain
xtrain = xtrain.values
ytrain = ytrain.values

xtest = TrainingDF.iloc[600:,1:] #Trims off ytrain
ytest = TrainingDF.iloc[600:,:1] #Trims off ytrain
xtest = xtest.values
ytest = ytest.values

print(xtrain.shape)
print(ytrain.shape)

print(xtest.shape)
print(ytest.shape)

(600, 11)
(600, 1)
(291, 11)
(291, 1)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [8]:
model = Sequential([
Dense(units = 512, activation="relu",input_shape=(11,)),
Dense(units = 512, activation="relu"),
Dense(units = 1, activation="sigmoid"),
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               6144      
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 1)                 513       
                                                                 
Total params: 269,313
Trainable params: 269,313
Non-trainable params: 0
_________________________________________________________________


2023-02-25 15:55:19.620522: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-25 15:55:19.624759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-25 15:55:19.624791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-25 15:55:19.625211: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other ope

In [9]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), metrics=[tf.keras.metrics.BinaryAccuracy()], optimizer="adam")

In [10]:
history = model.fit(
    xtrain, ytrain, epochs=10, verbose=1, validation_data=(xtest, ytest)
)

Epoch 1/10


2023-02-25 15:55:21.313822: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-25 15:55:21.318175: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f2b5c020b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-25 15:55:21.318211: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2023-02-25 15:55:21.321564: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-25 15:55:21.369768: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2023-02-25 15:55:21.369794: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:237] Used ptx

19/19 [==============================] - 1s 9ms/step - loss: 0.5342 - binary_accuracy: 0.7533 - val_loss: 0.4651 - val_binary_accuracy: 0.7973
Epoch 2/10
19/19 [==============================] - 0s 5ms/step - loss: 0.4611 - binary_accuracy: 0.8000 - val_loss: 0.4335 - val_binary_accuracy: 0.7973
Epoch 3/10
19/19 [==============================] - 0s 5ms/step - loss: 0.4428 - binary_accuracy: 0.8083 - val_loss: 0.4379 - val_binary_accuracy: 0.7938
Epoch 4/10
19/19 [==============================] - 0s 5ms/step - loss: 0.4358 - binary_accuracy: 0.8067 - val_loss: 0.4276 - val_binary_accuracy: 0.8007
Epoch 5/10
19/19 [==============================] - 0s 5ms/step - loss: 0.4295 - binary_accuracy: 0.8150 - val_loss: 0.4295 - val_binary_accuracy: 0.8041
Epoch 6/10
19/19 [==============================] - 0s 5ms/step - loss: 0.4250 - binary_accuracy: 0.8067 - val_loss: 0.4343 - val_binary_accuracy: 0.8110
Epoch 7/10
19/19 [==============================] - 0s 5ms/step - loss: 0.4272 - binary